In [11]:
import os
from dotenv import load_dotenv
import dtlpy as dl
from rfdetr.util.coco_classes import COCO_CLASSES



In [12]:
use_rc_env = False
if use_rc_env:
    dl.setenv('rc')
else:
    dl.setenv('prod')
if dl.token_expired():
    dl.login()


In [13]:
if use_rc_env:
    project = dl.projects.get(project_name='HusamDimo')
else:
    project = dl.projects.get(project_name='ShadiDemo')

In [14]:
models = project.models.list().print()



+----+--------------------------+-----------------------+----------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
# Publish your app
dpk = project.dpks.publish(local_path='model_adapter',manifest_filepath='dataloop.json')
# Install or update the application
try:
    app = project.apps.get(app_name=dpk.display_name)
    app.dpk_version = dpk.version
    app.update()
except dl.exceptions.NotFound:
    print("installing ...")
    app = project.apps.install(dpk=dpk)

Command Progress: 100%|██████████| 100/100 [00:02<00:00, 44.78it/s]


In [17]:
models = project.models.list().print()



+----+--------------------------+-----------------------+----------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
if use_rc_env:
    model_id = '67fe942e0bdaf97eb441f5c6'
else:
    model_id = '67fe3466f41fe3efebd2c433'
model = project.models.get(model_id=model_id)

set the lables from COCO_CLASSES

In [19]:
model.labels = list(COCO_CLASSES.values())

upload the weights file


In [20]:
print(f"model status before artifact upload: {model.status}")
# Upload your weights
artifact = model.artifacts.upload(filepath='model.pth')

# Update the configuration
model.configuration['weights_filename'] = artifact.filename


model = project.models.get(model_id=model_id)
print(f"model status after artifact upload: {model.status}")

model.status = 'trained'
model.update()

# # Deploy your model
print("-##################\n##################\n##################\n##################\n##################\n deploy only once \n##################\n##################\n##################\n##################\n##################\n")
model.deploy()

model = project.models.get(model_id=model_id)
print(f"model status after deploy: {model.status}")

model status before artifact upload: trained
Upload Items: 355MB [00:31, 11.8MB/s]                           
Iterate Entity: 100%|██████████| 1/1 [00:00<00:00,  6.67it/s]
model status after artifact upload: trained
-##################
##################
##################
##################
##################
 deploy only once 
##################
##################
##################
##################
##################

model status after deploy: deployed


In [22]:
dataset = project.datasets.get(dataset_name='nvidia-husam-clone-updated-name')
image_url = 'https://media.roboflow.com/notebooks/examples/dog-2.jpeg'
item_3 = dataset.items.upload(image_url)
prediction_3 = model.predict(item_ids=[item_3.id],dataset_id=dataset.id)
prediction_3.wait()
item_3.open_in_web()

Upload Items: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]
